# Model Evaluation

Now that we have trained the model, let's evaluate it against the test data.

In [1]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn

from src.backend.util.dataloader import get_data_loaders
from src.backend.model.model import CardClassifier

device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
# Load data
train_dir = '../data/train'
valid_dir = '../data/valid'
test_dir = '../data/test'
batch_size = 32
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

train_loader, valid_loader, test_loader, target_to_class = get_data_loaders(
    train_dir, valid_dir, test_dir, batch_size, transform
)

# Load model and criterion
model = CardClassifier()
model.load_state_dict(torch.load('../backend/model/saved_model/model_card_classifier.pth', map_location=device))
criterion = nn.CrossEntropyLoss()

In [9]:
test_loss = 0
correct = 0
total = 0

model.eval()
with torch.no_grad():  # Disable gradient computation
    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)  # Move data to the appropriate device
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, targets)  # Compute the loss

        test_loss += loss.item()  # Sum up batch loss
        _, predicted = torch.max(outputs.data, 1)  # Get the index of the max log-probability as the prediction
        total += targets.size(0)  # Total number of labels
        correct += (predicted == targets).sum().item()  # Correct predictions

# Calculate average losses and total accuracy
test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / total

print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{total} ({accuracy:.2f}%)')